In [1]:
import ee
from google.colab import drive
#drive.mount('data')
drive.mount('/content/drive')
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='abharajan')

import sys
sys.path.append('/content/drive/My Drive/BiomassApril2024')

import geemap

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import pandas as pd
import numpy as np

import utilceda as uc
import coordinates as cdn


Mounted at /content/drive


# Loop over all continents




In [ ]:
# Coordinate lists
cList = [cdn.clSEAsia, cdn.clAfrica, cdn.clSA, cdn.clNA, cdn.clEu]
area = ['SouthEastAsia', 'Africa', 'SouthAmerica', 'NorthAmerica', 'Europe']
nArea = len(area)

# Projection type
cciProjName = 'CCI'
s2ProjName = 'S2'
allProjName = 'general'
projectionType = 'S2' + s2ProjName + 'CCI' + cciProjName + 'All' + allProjName

# Plot Folder
plotFolder = 'drive/MyDrive/BiomassApril2024/EDAModelsAnalysis/' # reults go here

# columns against which Rsq will be plotted
xnamear = ['AGB', 'B1', 'B8', 'evi', 'ndvi', 'pSD']

xarar=[]
yar=[]

for iArea in range(0, nArea):

  # Folder locations
  dataFolder = 'drive/MyDrive/BiomassApril2024/EDAModelsAnalysis/'+ area[iArea] + '/' # input data folder

  # Coordinates list
  coordinatesList = cList[iArea]
  # No of coordinates
  ncoor = len(coordinatesList)

  # Place list
  placeList =[]
  for icoor in range(0, ncoor):
    place = area[iArea]  +'_' + str(round(coordinatesList[icoor][0],2)) + '_' +  str(round(coordinatesList[icoor][1],2))
    print(place)
    placeList.append(place)

  AGBs=[]
  B1s=[]
  B8s=[]
  evis=[]
  ndvis=[]
  psds=[]
  rsqXG=[]

  for i in range(0, ncoor):

    print(i)
    # Read results from file
    f = dataFolder + placeList[i] + projectionType
    EDAfile = f+'_EDA.dat' # EDA results
  #  corrfile = f+'_corr.dat'   # correlation matrix
    modelfile = f+'_ModelResults.dat' # model results

    EDAdf = uc.EDAModelFilestoDF(EDAfile)
  # corrdf = uc.EDAModelFilestoDF(corrfile)
    modeldf = uc.EDAModelFilestoDF(modelfile)

    AGBs.append([EDAdf.loc['mean','AGB'], EDAdf.loc['q50','AGB'], EDAdf.loc['rangeS','AGB'], EDAdf.loc['rangeL','AGB'], EDAdf.loc['var','AGB'] ])
    B1s.append([EDAdf.loc['mean','B1'], EDAdf.loc['q50','B1'], EDAdf.loc['rangeS','B1'], EDAdf.loc['rangeL','B1'], EDAdf.loc['var','B1'] ])
    B8s.append([EDAdf.loc['mean','B8'], EDAdf.loc['q50','B8'], EDAdf.loc['rangeS','B8'], EDAdf.loc['rangeL','B8'], EDAdf.loc['var','B8'] ])
    evis.append([EDAdf.loc['mean','evi'], EDAdf.loc['q50','evi'], EDAdf.loc['rangeS','evi'], EDAdf.loc['rangeL','evi'], EDAdf.loc['var','evi'] ])
    ndvis.append([EDAdf.loc['mean','ndvi'], EDAdf.loc['q50','ndvi'], EDAdf.loc['rangeS','ndvi'], EDAdf.loc['rangeL','ndvi'], EDAdf.loc['var','ndvi'] ])
    psds.append([EDAdf.loc['mean','pSD'], EDAdf.loc['q50','pSD'], EDAdf.loc['rangeS','pSD'], EDAdf.loc['rangeL','pSD'], EDAdf.loc['var','pSD'] ])
    rsqXG.append(modeldf.loc['rsq','XGBoost'])

  xarar.append([AGBs, B1s, B8s, evis, ndvis, psds])
  yar.append(rsqXG)

In [26]:
n_r = 2 # n_rows
n_c = 3

stats = ['mean', 'median', '50% range', '90% range', 'var']
nstats = len(stats)

colors=['blue', 'green', 'red', 'olive', 'purple']
markers=['o', '^', 'x', '<', 'D']

figs=[]
for istat in range(0, nstats):
  xstat=stats[istat]
  fig, axes = plt.subplots(n_r, n_c) # trends less

  for row_num in range(0, n_r):
    for col_num in range(0, n_c):
      icol = n_c*row_num + col_num
      ax = axes[row_num][col_num]

      for iArea in range(0, nArea):
        xar = xarar[iArea]
        ax.scatter(pd.DataFrame(np.array(xar[icol])).iloc[:,istat], yar[iArea], s=50, c=colors[iArea], marker=markers[iArea],
                   label=area[iArea], alpha = 0.7)

      ax.tick_params(axis='x', labelsize=15)
      ax.tick_params(axis='y', labelsize=15)
      ax.set_title(xnamear[icol], fontsize=15)
      if icol == 5:
        ax.legend(fontsize=15)

  fig.set_size_inches(15, 10)
  fig.suptitle('rsqXG vs '+ xstat, x=0.5, y=0.95, fontsize=20)
  fig.tight_layout(pad=3)

  figs.append(fig)

# Save figures to file
pdfFile = plotFolder + 'ResultsRsqStats' + '.pdf'
p = PdfPages(pdfFile)

for istat in range(0, nstats):
  figs[istat].savefig(p, format='pdf')

p.close()

# Close plots
for istat in range(0, nstats):
  plt.close(figs[istat])

# One Area

In [ ]:
# Set coordinates to choose datasets

cList = [cdn.clIndia, cdn.clSEAsia, cdn.clAfrica, cdn.clSA, cdn.clNA, cdn.clEu]
area = ['India', 'SouthEastAsia', 'Africa', 'SouthAmerica', 'NorthAmerica', 'Europe']

iArea = 4

# Coordinates list
coordinatesList = cList[iArea]
# No of coordinates
ncoor = len(coordinatesList)

# Place list
#placeList = cdn.plIndia
placeList =[]
for icoor in range(0, ncoor):
  place = area[iArea]  +'_' + str(round(coordinatesList[icoor][0],2)) + '_' +  str(round(coordinatesList[icoor][1],2))
  print(place)
  placeList.append(place)

# coordinatesList = [[38.99458221563848, 6.463386197712386]]
# placeList = ['Africa' ]

# columns against which Rsq will be plotted
lesscol = ['AGB', 'B1', 'B8', 'evi', 'ndvi', 'pSD']


# Projection type
cciProjName = 'CCI'
s2ProjName = 'S2'
allProjName = 'general'
projectionType = 'S2' + s2ProjName + 'CCI' + cciProjName + 'All' + allProjName

# Folder locations
dataFolder = 'drive/MyDrive/BiomassApril2024/EDAModelsAnalysis/'+ area[iArea] + '/' # input data folder
plotFolder = 'drive/MyDrive/BiomassApril2024/EDAModelsAnalysis/' # reults go here

NorthAmerica_9.42_44.7
NorthAmerica_4.39_44.71
NorthAmerica_15.8_47.97
NorthAmerica_23.18_49.08
NorthAmerica_22.58_48.83
NorthAmerica_23.91_48.69
NorthAmerica_24.93_62.47
NorthAmerica_22.18_61.65
NorthAmerica_28.5_63.06
NorthAmerica_15.75_61.62
NorthAmerica_14.6_57.5
NorthAmerica_28.91_58.41


In [ ]:
# Analysis summary for each coordinate

AGBs=[]
B1s=[]
B8s=[]
evis=[]
ndvis=[]
psds=[]
rsqXG=[]

for i in range(0, ncoor):

  print(i)
  # Read results from file
  f = dataFolder + placeList[i] + projectionType
  EDAfile = f+'_EDA.dat' # EDA results
#  corrfile = f+'_corr.dat'   # correlation matrix
  modelfile = f+'_ModelResults.dat' # model results

  EDAdf = uc.EDAModelFilestoDF(EDAfile)
 # corrdf = uc.EDAModelFilestoDF(corrfile)
  modeldf = uc.EDAModelFilestoDF(modelfile)

  AGBs.append([EDAdf.loc['mean','AGB'], EDAdf.loc['q50','AGB'], EDAdf.loc['rangeS','AGB'], EDAdf.loc['rangeL','AGB'], EDAdf.loc['var','AGB'] ])
  B1s.append([EDAdf.loc['mean','B1'], EDAdf.loc['q50','B1'], EDAdf.loc['rangeS','B1'], EDAdf.loc['rangeL','B1'], EDAdf.loc['var','B1'] ])
  B8s.append([EDAdf.loc['mean','B8'], EDAdf.loc['q50','B8'], EDAdf.loc['rangeS','B8'], EDAdf.loc['rangeL','B8'], EDAdf.loc['var','B8'] ])
  evis.append([EDAdf.loc['mean','evi'], EDAdf.loc['q50','evi'], EDAdf.loc['rangeS','evi'], EDAdf.loc['rangeL','evi'], EDAdf.loc['var','evi'] ])
  ndvis.append([EDAdf.loc['mean','ndvi'], EDAdf.loc['q50','ndvi'], EDAdf.loc['rangeS','ndvi'], EDAdf.loc['rangeL','ndvi'], EDAdf.loc['var','ndvi'] ])
  psds.append([EDAdf.loc['mean','pSD'], EDAdf.loc['q50','pSD'], EDAdf.loc['rangeS','pSD'], EDAdf.loc['rangeL','pSD'], EDAdf.loc['var','pSD'] ])
  rsqXG.append(modeldf.loc['rsq','XGBoost'])




0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
n_r = 2 # n_rows
n_c = 3

xar=[AGBs, B1s, B8s, evis, ndvis, psds]
xnamear=['AGB', 'B1', 'B8', 'evi', 'ndvi', 'pSD']

stats = ['mean', 'median', '50% range', '90% range', 'var']
nstats = len(stats)

figs=[]
for istat in range(0, nstats):
  xstat=stats[istat]
  fig, axes = plt.subplots(n_r, n_c) # trends less

  for row_num in range(0, n_r):
    for col_num in range(0, n_c):
      icol = n_c*row_num + col_num
      ax = axes[row_num][col_num]
      ax.scatter(pd.DataFrame(np.array(xar[icol])).iloc[:,istat], rsqXG, s=30, marker='o')
      ax.set_title(xnamear[icol])

  fig.set_size_inches(15, 10)
  fig.suptitle('rsqXG vs '+ xstat, fontsize=12)
  fig.tight_layout(pad=1)

  figs.append(fig)

# Save figures to file
pdfFile = plotFolder + 'ResultsRsqStats' + '.pdf'
p = PdfPages(pdfFile)

for istat in range(0, nstats):
  figs[istat].savefig(p, format='pdf')

p.close()

# Close plots
for istat in range(0, nstats):
  plt.close(figs[istat])


In [ ]:
print(pd.DataFrame(np.array(AGBs)))
print(rsqXG)

           0         1         2         3             4
0    46.0702   23.3997   58.1557  161.7673   2942.215653
1    72.6058   61.5778   82.4181  178.0216   3157.558570
2   181.0003  194.2597  196.4503  341.7334  12585.262636
3   191.8288  203.4010  177.8864  357.9095  12651.315553
4   200.5219  198.7910  224.3278  421.9034  17994.863299
5   134.7736  141.2998  120.3659  256.7838   6378.445493
6    77.4051   82.0076   64.4458  123.3829   1523.551545
7    67.9808   68.0899   66.4250  124.5915   1562.291673
8    80.6900   86.9470   58.2927  116.3450   1332.609923
9    68.3008   70.3253   68.4193  122.2283   1554.141579
10   77.7644   79.7495   73.4331  135.0563   1865.651362
11  157.0522  165.0912  107.1056  248.0426   5471.013368
[0.4742020535800259, 0.538450444603839, 0.6351921162302243, 0.6229500307982493, 0.604445800319181, 0.6555194271779949, 0.6006373189349512, 0.6288455924021616, 0.6661077191375373, 0.7531808116351559, 0.7057707228063798, 0.6229100790726387]
